<a href="https://colab.research.google.com/github/Sahil-Chhabra-09/Review-Classification/blob/main/KaggleWars_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

## Importing data

In [9]:
train = pd.read_csv("/content/train (1).csv")

## Understanding data

In [10]:
train.shape

(21000, 2)

In [11]:
train.head()

,Review,Rating
0,Great game when i was a kid I grew up watching...,5
1,Nice game,1
2,Balo game but bholo laguchi khelibaku gote khu...,5
3,This is the worst computer cheating game I've ...,1
4,Great,5


In [12]:
train.Rating.value_counts()

5    12376
1     4120
4     2233
3     1369
2      902
Name: Rating, dtype: int64

In [13]:
train.isna().sum()

Review    0
Rating    0
dtype: int64

## Counting frequency of each word to find outliers

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
# from collections import Counter
# cnt = Counter()

# for text in train.Review:
#   for word in text.split():
#     if word not in stopwords_english:
#       cnt[word]+=1

In [16]:
# cnt.most_common()[::-1]

In [17]:
# custom_stopwords = []
# for word in cnt:
#   if cnt[word]<2:
#     custom_stopwords.append(word)
#   if cnt[word]>7000:
#     custom_stopwords.append(word)

In [18]:
# len(custom_stopwords)

In [19]:
# len(stopwords_english)

In [20]:
# stopwords_english.extend(custom_stopwords)

In [21]:
# len(stopwords_english)

## Preprocessing

In [22]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [23]:
train

,Review,Rating
0,Great game when i was a kid I grew up watching...,5
1,Nice game,1
2,Balo game but bholo laguchi khelibaku gote khu...,5
3,This is the worst computer cheating game I've ...,1
4,Great,5
...,...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,5
20996,Very nice and enjoy!!!,4
20997,It has lots of different games depending on yo...,5
20998,Has several very frustrating bugs not present ...,3


In [24]:
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
import re
def process_review(text):
  stemmer = PorterStemmer()
  # text = re.sub(r'\b\w\b', ' ', text)
  # text = re.sub("[ \d+| \d+ ]",' ',text)
  # text = re.sub(r"\.+",' ', text)
  text = text.lower()
  text_tokens = word_tokenize(text)
  text_clean = []
  for word in text_tokens:
    if (word not in stopwords_english and word not in string.punctuation):
      stem_word = stemmer.stem(word)
      text_clean.append(word)
  return text_clean

In [26]:
process_review(train.Review[0])[:15]

['great',
 'game',
 'kid',
 'grew',
 'watching',
 'pokémon',
 'buying',
 'pokemon',
 'cards',
 'kid',
 'play',
 'catch',
 'pokémon',
 'saw',
 'tv']

## Creating a dictionary which contains key as: (word, review) and value as their frequency

In [27]:
import numpy as np

In [28]:
def build_freqs(all_reviews, ratings):
  rat_list = np.squeeze(ratings).tolist()
  freqs = {}
  for rat,review in zip(rat_list, all_reviews):
    for word in process_review(review):
      pair = (word, rat)
      if(pair in freqs):
        freqs[pair]+=1
      else:
        freqs[pair] = 1
  return freqs

In [29]:
freqs = build_freqs(train.Review, train.Rating)

In [30]:
# freqs

## Creating a dataframe for logistic regression model

In [31]:
df = pd.DataFrame(columns = ['bias', 'one', 'two', 'three', 'four', 'five', 'rating'])

In [32]:
train.iloc[0].Rating

5

In [33]:
for i,review in enumerate(train.Review):
  lst = [3.85]
  one = 0
  two = 0
  three = 0
  four = 0
  five = 0
  for word in process_review(review):
    one = one + freqs.get((word, 1), 0)
    two = two + freqs.get((word, 2), 0)
    three = three + freqs.get((word, 3), 0)
    four = four + freqs.get((word, 4), 0)
    five = five + freqs.get((word, 5), 0)
  lst.append(one)
  lst.append(two)
  lst.append(three)
  lst.append(four)
  lst.append(five)
  lst.append(train.iloc[i].Rating)
  df.loc[len(df.index)] = lst

In [34]:
df.shape

(21000, 7)

In [35]:
df.head(10)

,bias,one,two,three,four,five,rating
0,3.85,9691.0,2483.0,3733.0,4771.0,18757.0,5.0
1,3.85,2670.0,704.0,1144.0,1711.0,8086.0,1.0
2,3.85,2613.0,670.0,1087.0,1548.0,7087.0,5.0
3,3.85,8140.0,1994.0,3012.0,3975.0,16791.0,1.0
4,3.85,80.0,27.0,89.0,226.0,782.0,5.0
5,3.85,883.0,381.0,755.0,1175.0,3655.0,4.0
6,3.85,40.0,12.0,38.0,41.0,399.0,5.0
7,3.85,92.0,51.0,114.0,195.0,1641.0,5.0
8,3.85,2633.0,670.0,1093.0,1548.0,7194.0,5.0
9,3.85,980.0,167.0,183.0,300.0,1146.0,1.0


In [36]:
df.rating.value_counts()

5.0    12376
1.0     4120
4.0     2233
3.0     1369
2.0      902
Name: rating, dtype: int64

## Balancing the imbalanced data

In [37]:
X = df[['bias', 'one', 'two', 'three','four', 'five']].values
Y = df['rating'].values

In [38]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(X, Y)

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_smote,y_smote, test_size = 0.15, random_state = 0)

## Standard scaling data

In [40]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# scaler.fit(x_smote)
# x_smote = scaler.transform(x_smote)

In [41]:
# x_test = scaler.transform(x_test)

## Training our Random Forest model

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 201)
rf.fit(x_smote, y_smote)

RandomForestClassifier(n_estimators=1000, random_state=201)

In [44]:
score = rf.score(x_test, y_test)
print("Score of model: ")
print(score)

Score of model: 
0.9489334195216548


## Trying random hyperparameters for random forest

In [45]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth}
# print(random_grid)

## Training our logistic regression model

In [46]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix

# # model = LogisticRegression(random_state= 0, solver = 'lbfgs', max_iter = 2000).fit(x_smote, y_smote)
# model = LogisticRegression(random_state= 10).fit(x_smote, y_smote)
# score = model.score(x_test, y_test)
# print("Score of model: ")
# print(score)

In [47]:
# y_actual = pd.Series(y_test, name='Actual')
# y_predicted = pd.Series(model.predict(x_test), name='Predicted')
# print("Confusion matrix: ")
# pd.crosstab(y_actual,y_predicted)

In [48]:
# temp_df = pd.DataFrame(model.predict(x_test))

In [49]:
# temp_df

In [50]:
# temp_df.value_counts()

## Predicting our testing data

In [51]:
test = pd.read_csv("/content/test (2).csv")

In [52]:
test.shape

(7000, 1)

In [53]:
test.head()

,Review
0,This used to be my favorite game and it is sti...
1,Every single time i put the online multi-playe...
2,Determined to make us watch ads! Wouldn't be a...
3,Super great running game
4,"Hello NetEase, I really loved the game it dese..."


In [54]:
test.Review

0       This used to be my favorite game and it is sti...
1       Every single time i put the online multi-playe...
2       Determined to make us watch ads! Wouldn't be a...
3                                Super great running game
4       Hello NetEase, I really loved the game it dese...
                              ...                        
6995                   This is like playing real football
6996                              Please fix aim accuracy
6997    Fix your game ranking and chat, this game suck...
6998                                    give coins 500000
6999                                            Good game
Name: Review, Length: 7000, dtype: object

## Generating Xm for test data

In [55]:
f_lst = []

In [56]:
for review in test.Review:
  lst = [3.85]
  one = 0
  two = 0
  three = 0
  four = 0
  five = 0
  for word in process_review(review):
    one = one + freqs.get((word, 1), 0)
    two = two + freqs.get((word, 2), 0)
    three = three + freqs.get((word, 3), 0)
    four = four + freqs.get((word, 4), 0)
    five = five + freqs.get((word, 5), 0)
  lst.append(one)
  lst.append(two)
  lst.append(three)
  lst.append(four)
  lst.append(five)
  f_lst.append(lst)

In [57]:
test_df = pd.DataFrame(f_lst, columns = ['bias', 'one', 'two', 'three', 'four', 'five'])

In [58]:
test_df.head()

,bias,one,two,three,four,five
0,3.85,2893,806,1208,1677,7465
1,3.85,2493,597,880,875,1711
2,3.85,12769,3609,4947,5936,20061
3,3.85,2750,719,1197,1801,8112
4,3.85,5979,1647,2707,3761,15644


In [59]:
test_x = test_df[['bias', 'one', 'two', 'three', 'four', 'five']].values

In [60]:
# scaler = StandardScaler()

# scaler.fit(test_x)
# test_x = scaler.transform(test_x)

In [61]:
rf.predict(test_x)

array([5., 1., 2., ..., 1., 5., 5.])

In [62]:
test_res = pd.DataFrame(rf.predict(test_x))

In [63]:
test_res.to_csv('./outran17.csv')